# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [115]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

In [116]:
import numpy as np 
import pandas as pd
import requests 
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [117]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [118]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [119]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}' 
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 85853191, 'calculationPrice': 'close', 'change': 0.7, 'changePercent': 0.00474, 'close': 149.69, 'closeSource': 'aflociif', 'closeTime': 1729589866800, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 147.629, 'delayedPriceTime': 1667506631173, 'extendedChange': -0.28, 'extendedChangePercent': -0.0019, 'extendedPrice': 153.52, 'extendedPriceTime': 1719634777801, 'high': 154.31, 'highSource': 'uye1ddi5inet le caermp ', 'highTime': 1699922960694, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 151.21, 'iexCloseTime': 1684954155012, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 148.03, 'iexOpenTime': 1692650207220, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1713049195384, 'latestPrice': 153.23, 'latestSource': 'Close', 'latestTime': 'July 8, 2022', 'latestUpdate': 1730430064025, 'latestVolume': 66032456, 'low': 145.71, 'lowSource': ' r iilptnyee1emed5c 

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [120]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap)

2458697562800


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [121]:
my_columns = ['ticker', 'price', 'market_cap', 'num_to_buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,ticker,price,market_cap,num_to_buy


In [122]:
final_dataframe.append(pd.Series([symbol, price, market_cap, 'N/A'], index=my_columns), ignore_index=True)

,ticker,price,market_cap,num_to_buy
0,AAPL,153.23,2458697562800,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [123]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(pd.Series([symbol, data['latestPrice'], data['marketCap'], 'N/A'], index=my_columns), ignore_index=True)

final_dataframe

,ticker,price,market_cap,num_to_buy
0,AAPL,127.65,38256737234,N/A
1,AAPL,14.29,9444011993,N/A
2,AAPL,183.3,11312574413,N/A
3,AAPL,149.33,2449774250229,N/A
4,AAPL,155.78,277395940576,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [124]:
def chunks(lst, n): 
    """Yield successive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [125]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [126]:
print(symbol_strings[:1])

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA']


In [127]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings: 
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        try: 
            final_dataframe = final_dataframe.append(
                pd.Series(
                [   
                symbol,
                data[symbol]['quote']['latestPrice'], 
                data[symbol]['quote']['marketCap'], 
                'N/A'   
                ], 
                index = my_columns
                ),
                ignore_index=True
            )
        except: 
                print(f'{symbol} is not part of S&P500')

DISCA is not part of S&P500
HFC is not part of S&P500
VIAC is not part of S&P500
WLTW is not part of S&P500


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [128]:
portfolio_size = input('Enter the value of your portfolio: ')

try: 
    val = float(portfolio_size)
except ValueError: 
    print('\nThat is not a number. No commas allowed.')

Enter the value of your portfolio: 10e6


In [129]:
final_dataframe.shape

(501, 4)

In [130]:
final_dataframe.tail()

,ticker,price,market_cap,num_to_buy
496,YUM,122.48,34842078883,N/A
497,ZBH,108.69,23060806360,N/A
498,ZBRA,311.39,15804576171,N/A
499,ZION,51.2,7889897874,N/A
500,ZTS,186.03,84393546046,N/A


In [131]:
final_dataframe.loc[:, 'price']

0      126.27
1          14
2       188.4
3      148.22
4      154.67
        ...  
496    122.48
497    108.69
498    311.39
499      51.2
500    186.03
Name: price, Length: 501, dtype: object

In [132]:
position_size = val / final_dataframe.shape[0]

for i in range(0, final_dataframe.shape[0]): 
    final_dataframe.loc[i, 'num_to_buy'] = math.floor(position_size / final_dataframe.loc[i, 'price'])

final_dataframe

,ticker,price,market_cap,num_to_buy
0,A,126.27,36996434645,158
1,AAL,14,9330131829,1425
2,AAP,188.4,11238286434,105
3,AAPL,148.22,2391081273851,134
4,ABBV,154.67,280488718752,129
...,...,...,...,...
496,YUM,122.48,34842078883,162
497,ZBH,108.69,23060806360,183
498,ZBRA,311.39,15804576171,64
499,ZION,51.2,7889897874,389


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [133]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

 

In [134]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color, 
        'bg_color': background_color, 
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00', 
        'font_color': font_color, 
        'bg_color': background_color, 
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color, 
        'bg_color': background_color, 
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [135]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()

# writer.sheets['Recommended Trades'].write('A1', 'ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'price', dollar_format)
# writer.sheets['Recommended Trades'].write('C1', 'market_cap', dollar_format)
# writer.sheets['Recommended Trades'].write('D1', 'num_to_buy', integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [136]:
column_formats = { 
    'A': ['ticker', string_format], 
    'B': ['price', dollar_format], 
    'C': ['market_cap', dollar_format],
    'D': ['num_to_buy', integer_format]
}

for column in column_formats.keys(): 
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [137]:
writer.save()